In [1]:
import parser 
import pandas as pd
import itertools
import numpy as np
import os 
import matplotlib.pyplot as plt
import seaborn as sns
from voting_rules import voting_rules as vr
import rcv_dimensionality

In [ ]:
def is_consistent2(ballot, perm):

    if len(ballot) == 0:
        return True
    c = ballot[0]
    index = 0
    for i in range(len(perm)):
        if (c == perm[i]):
            index = i
            break
    def check(i, j, p):
        '''
        i: index in the ballot
        j: index in the permutation
        p = parent 
        '''
        if j < 0 or j > len(perm):
            return False
        
        if (i == len(ballot) - 1):
            return True
        
        if (i == 0):
            if (j + 1 >= len(perm)):
                if ballot[i + 1] != perm[j - 1]:
                    return False 
            if j - 1 < 0:
                if ballot[i + 1] != perm[j + 1]:
                    return False
            if ballot[i + 1] != perm[j - 1] and ballot[i + 1] != perm[j + 1]:
                return False

        if p != j + 1 and j + 1 < len(perm) and ballot[i + 1] == perm[j + 1]:
            return check(i + 1, j + 1, j)
        elif p != j - 1 and ballot[i + 1] == perm[j - 1]:
            return check(i + 1, j - 1, j)
        else:
            if p != j:
                return check(i, p, p)
            else:
                return False
    return check(0, index, -1)


In [2]:
def is_consistent(ballot):
    if len(ballot) == 0 or len(ballot) == 1:
        return True
    x = 0 
    v = 0.25
    i = 1
    while (i < len(ballot)):
        if ballot[i] < ballot[i - 1]:
            x -= (v * min(abs(ballot[i] - ballot[0]), abs(ballot[i] - ballot[i - 1])))
        else:
            x += (v * min(abs(ballot[i] - ballot[0]), abs(ballot[i] - ballot[i - 1])))
        v *= 0.5
        i += 1
    l = []
    if abs(x) >= 0.5 or x == 0:
        return False 
    x += ballot[0]
    for c in ballot:
        l.append(abs(c - x))
    if(all(l[i] <= l[i + 1] for i in range(len(l) -  1))):
        return True 
    return False

In [7]:
def dim(ballots, candidates):
    perms = list(itertools.permutations(candidates))
    #most consistant permutation
    mcp = None
    c_mcp = 0
    x = len(candidates)
    l = [i for i in range(1, x + 1)]
    for perm in perms:
        temp = {}
        i = 1
        for candidate in perm:
            temp[candidate] = i
            i += 1
        c = 0
        for b in ballots:
            b_num = []
            for candidate in b:
                b_num.append(temp[candidate])
            if (is_consistent(b_num)):
                c += ballots[b]
        if mcp is None or c > c_mcp:
            c_mcp = c
            mcp = perm
    return (c_mcp/sum(ballots.values())), mcp

In [ ]:
def dimensionality_analysis(directory, filename):
    
    
    

    
    

    if len(candidates) <= 7:
        gamma, mcp = dim(ballots, candidates)
        
    
    #print(df)
    #print(df7)
    return df


In [ ]:
matched_elections = pd.read_csv("dataverse_files_partisan 2.csv")
row_num = matched_elections[matched_elections["filename"] == "Alaska_08162022_HouseofRepresentativesSpecial.csv"].index[0]
print(row_num)
print(matched_elections.loc[row_num, "type"])

In [4]:
def get_gamma(mcp, mcp_num, ballots, candidates):

    perms = itertools.permutations(candidates)
    c_mcp = 0
    x = len(candidates)
    
    temp = {}
    for i in range(len(mcp)):
        temp[mcp[i]] = mcp_num[i]
    c = 0
    for b in ballots:
        b_num = []
        for candidate in b:
            if candidate in mcp:
                b_num.append(temp[candidate])
        if (is_consistent(b_num)):
            c += ballots[b]
    
    return c/sum(ballots.values())

In [7]:
directory = "dataverse_files"
matched_elections = pd.read_csv("dataverse_files_partisan 2.csv")
df = pd.DataFrame(columns=["Election", "candidates", "Partisan", "Gamma", "Most Consistent Permutaition"])

for filename in os.listdir(directory):
    
    ballots, candidates = parser.parser(os.path.join(directory, filename))
    election = vr(ballots, candidates)
    row_num = matched_elections[matched_elections["filename"] == filename].index[0]
    s = "filename==" + "'" + filename + "'"
    
    if len(candidates) < 3:
        matched_elections.loc[row_num, "gamma"] = 1
        continue 
    try:
        d = rcv_dimensionality.perform_rcv_and_normalize(os.path.join(directory, filename))
    except:
        print(filename)
    mcp = []
    mcp_num = []
    for k in d:
        mcp.append(k)
        mcp_num.append(d[k])
    
    gamma = get_gamma(mcp, mcp_num, ballots, candidates)
    
    matched_elections.loc[row_num, "gamma"] = gamma

    
    '''matched_elections.loc[row_num, "candidates"] = len(candidates)
    try:
        matched_elections.loc[row_num, "irv"] = election.irv()
    except:
        print(filename, "failed irv")
    
    try:
        matched_elections.loc[row_num, "condorcet"] = election.condorcet()
    except:
        print(filename, " failed condorcet")
        
    try:
        matched_elections.loc[row_num, "approval"] = election.approval()
    except:
        print(filename, " failed ", "approval")
    
    try:
        matched_elections.loc[row_num, "plurality"] = election.plurality()
    except:
        print(filename, " failed plurality")'''

   
    
print(matched_elections)
    


/Users/Mahshad/Documents/GitHub/bugs-in-democracy/team_arrow/cvr_class/rcv_dimensionality.py:163: RuntimeWarning: divide by zero encountered in divide
  distance = 1 / np.sqrt(freq_upper_triangle)
/Users/Mahshad/Documents/GitHub/bugs-in-democracy/team_arrow/cvr_class/rcv_dimensionality.py:163: RuntimeWarning: divide by zero encountered in divide
  distance = 1 / np.sqrt(freq_upper_triangle)
/Users/Mahshad/Documents/GitHub/bugs-in-democracy/team_arrow/cvr_class/rcv_dimensionality.py:163: RuntimeWarning: divide by zero encountered in divide
  distance = 1 / np.sqrt(freq_upper_triangle)
/Users/Mahshad/Documents/GitHub/bugs-in-democracy/team_arrow/cvr_class/rcv_dimensionality.py:163: RuntimeWarning: divide by zero encountered in divide
  distance = 1 / np.sqrt(freq_upper_triangle)
/Users/Mahshad/Documents/GitHub/bugs-in-democracy/team_arrow/cvr_class/rcv_dimensionality.py:163: RuntimeWarning: divide by zero encountered in divide
  distance = 1 / np.sqrt(freq_upper_triangle)
/Users/Mahshad/

KeyError: 'Seth Grimes]WRITE-IN114'

In [8]:
matched_elections.to_csv("gamma.csv")

In [10]:
print(matched_elections)
row_num = matched_elections[matched_elections["filename"] == "Easthampton_11022021_Mayor.csv"].index[0]
matched_elections.loc[row_num, "candidates"] = len(candidates)
try:
    matched_elections.loc[row_num, "irv"] = election.irv()
except:
    print(filename, "failed irv")

try:
    matched_elections.loc[row_num, "condorcet"] = election.condorcet()
except:
    print(filename, " failed condorcet")
    
try:
    matched_elections.loc[row_num, "approval"] = election.approval()
except:
    print(filename, " failed ", "approval")

try:
    matched_elections.loc[row_num, "plurality"] = election.plurality()
except:
    print(filename, " failed plurality")
matched_elections.to_csv("matched.csv")

                                              filename partisan  level type  \
0       Alaska_04102020_PRESIDENTOFTHEUNITEDSTATES.csv       DP    3.0    E   
1    Alaska_08162022_HouseofRepresentativesSpecial.csv        1    3.0    R   
2       Alaska_11082022_GovernorLieutenantGovernor.csv        1    2.0    E   
3                   Alaska_11082022_HouseDistrict1.csv        1    2.0    R   
4                   Alaska_11082022_HouseDistrict2.csv        1    2.0    R   
..                                                 ...      ...    ...  ...   
439                      TakomaPark_11082022_Mayor.csv        0    1.0    E   
440                        Vineyard_11022021_Mayor.csv        0    1.0    E   
441                  Vineyard_11052019_CityCouncil.csv        0    1.0    R   
442                   WoodlandHills_11022021_Mayor.csv        0    1.0    E   
443    Wyoming_04172020_PRESIDENTOFTHEUNITEDSTATES.csv      NaN    3.0    E   

     candidates     gamma  irv   condorcet         

In [10]:
t = 0
n = 0
no_condorcet = 0
ballots, candidates = parser.parser("dataverse_files/Alaska_11082022_HouseDistrict11.csv")
election = vr(ballots, candidates)
print(election.condorcet())

NameError: name 'dbfbfs' is not defined

In [12]:
t = 0
n = 0
no_condorcet = 0
for i, row in matched_elections.iterrows():
    if row['condorcet'] == -1:
        no_condorcet += 1
    if row['irv'] == row["condorcet"]:
        n += 1
    t += 1
print(no_condorcet)
print(n, " ", t)
print(n/t)

2
439   444
0.9887387387387387


In [ ]:
print(no_condorcet)

In [6]:
ballots, candidates = parser.parser("dataverse_files/Alaska_11082022_HouseDistrict11.csv")
election = vr(ballots, candidates)
election.condorcet()

NameError: name 'dbfbfs' is not defined

In [ ]:
def plot(ballots, candidates, perm):
    temp = {}
    i = 1
    for c in perm:
        temp[c] = i
        i += 1

In [ ]:
df = dimensionality_analysis("dataverse_files")
print(df.head())

In [ ]:
print(df.head())
df.to_csv("gamma.csv")

In [ ]:
def dimensionality_analysis_primary(directory):
    
    matched_elections = pd.read_csv("dataverse_files_partisan 2.csv")
    df = pd.DataFrame(columns=["Election", "candidates", "Party", "Gamma", "Most Consistent Permutaition"])
    df7 = pd.DataFrame(columns=["Election", "candidates", "Party", "Gamma", "Most Consistent Permutaition"])
    party  = "D"
    for filename in os.listdir(directory):
        s = "filename==" + "'" + filename + "'"
        try:
            #print(str(matched_elections.query(s)["partisan"].tolist()[0]))
            if str(matched_elections.query(s)["partisan"].tolist()[0]) == "DP":
                party = "D"
            elif str(matched_elections.query(s)["partisan"].tolist()[0]) == "RP":
                party = "R"
            else:
                continue
                
        except:
            continue
        
        
        ballots, candidates = parser.parser(os.path.join(directory, filename))
        
        
        #print(s)

        if len(candidates) <= 7:
            gamma, mcp = dim(ballots, candidates, party)
            df.loc[len(df)] = [filename, len(candidates), party, gamma, mcp]
            if len(candidates) == 7:
                df7.loc[len(df7)] = [filename, len(candidates), party, gamma, mcp]
                print("filename ", filename, "Partisan: ", party)
                print("mcp: ", mcp)
                print("gamma: ", gamma)
                print(ballots)
                print()
    
    #print(df)
    #print(df7)
    return df


In [ ]:
primary_df = dimensionality_analysis_primary("dataverse_files")



In [ ]:
print(primary_df)

In [ ]:
d_D = {}
d_R = {}
min_gamma = None
min_row = None 
for i, row in primary_df.iterrows():
    if row["Party"] == "D":
        if row["candidates"] not in d_D:
            d_D[row["candidates"]] = []
        d_D[row["candidates"]].append(row["Gamma"])
        if min_gamma is None or row["Gamma"] < min_gamma:
            min_gamma = row["Gamma"]
            min_row = row
    
    if row["Party"] == "R":
        if row["candidates"] not in d_R:
            d_R[row["candidates"]] = []
        d_R[row["candidates"]].append(row["Gamma"])

print(min_row)
print(min_row.loc["Election"])

In [ ]:
#plotting the democratic primaries

plt.figure(figsize=(10, 6))

for x, y_values in d_D.items():
    for y in y_values:
        plt.scatter(x, y, color='b')

In [ ]:
#plotting the republican primaries 

plt.figure(figsize=(10, 6))

for x, y_values in d_R.items():
    
    for y in y_values:
        plt.scatter(x, y, color='r')

In [ ]:
ballots, candidates = parser.parser("dataverse_files/Alaska_04102020_PRESIDENTOFTHEUNITEDSTATES.csv")
perms = itertools.permutations(candidates)
mcp = None
c_mcp = None
for perm in perms:
    temp = {}
    i = 1
    for candidate in perm:
        temp[candidate] = i
        i += 1
    c = 0
    for b in ballots:
        b_num = []
        for candidate in b:
            b_num.append(temp[candidate])
        if (is_consistent(b_num)):
            c += ballots[b]
    if mcp is None or c > c_mcp:
        c_mcp = c
        mcp = perm
    if perm == ('Guillermo A. Perez','Ben Kallos', 'Elizabeth R. Caputo', 'Mark D. Levine','Lindsey C. Boylan', 'Brad M. Hoylman', 'Kimberly R. Watkins'):
        print(perm, "  ", c/sum(ballots.values()))
    #print(perm, "   ", c/sum(ballots.values()))
print()
print(mcp, "   ", c_mcp/sum(ballots.values()))


In [ ]:
d_p = {}
d_nonp = {}
for i, row in df.iterrows():
    if row["Partisan"] == True:
        if row["candidates"] not in d_p:
            d_p[row["candidates"]] = []
        d_p[row["candidates"]].append(row["Gamma"])
    if row["Partisan"] == False:
        if row["candidates"] not in d_nonp:
            d_nonp[row["candidates"]] = []
        d_nonp[row["candidates"]].append(row["Gamma"])

sns.violinplot(data=df, x="candidates", y="Gamma", hue="Partisan", split=True)

In [ ]:
plt.figure(figsize=(10, 6))

for x, y_values in d_p.items():
    for y in y_values:
        plt.scatter(x, y, color='r')

"""for x, y_values in d_nonp.items():
    for y in y_values:
        plt.scatter(x, y, color='b')"""
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))

'''for x, y_values in d_p.items():
    for y in y_values:
        plt.scatter(x, y, color='r')'''

for x, y_values in d_nonp.items():
    for y in y_values:
        plt.scatter(x, y, color='b')
plt.show()

In [ ]:
p_counter = 0 
for n in d_p:
    p_counter += len(d_p[n])
nonp_counter = 0 
for n in d_nonp:
    nonp_counter += len(d_nonp[n])
print(p_counter)
print(nonp_counter)
print(p_counter+nonp_counter)

In [ ]:
# d1: partisan, d2: nonpartisan
d1, d2 = dim()

In [ ]:
plt.figure(figsize=(10, 6))

for x, y_values in d1.items():
    for y in y_values:
        if x != 0:
            plt.scatter(x, y, color='r')

In [ ]:
plt.figure(figsize=(10, 6))

'''for x, y_values in d_p.items():
    for y in y_values:
        plt.scatter(x, y, color='r')'''

for x, y_values in d2.items():
    for y in y_values:
        plt.scatter(x, y, color='b')
plt.show()

In [ ]:


#df = pd.DataFrame(columns=["candidates", "gamma", "partisan"])
'''for x, y_values in d_p.items():
    for y in y_values:
        df.loc[len(df)] = [x, y, True]
for x, y_values in d2.items():
    for y in y_values:
        df.loc[len(df)] = [x, y, False]
df.to_csv("gamma.csv", index=False)'''

In [ ]:
sns.violinplot(data=df, x="candidates", y="gamma", hue="partisan", split=True)

In [ ]:
import seaborn as sns

filename = "dataverse_files/Maine_11062018_CongressionalDistrict2.csv"
ballots, candidates = parser.parser(filename)

perms = list(itertools.permutations(candidates))
#most consistant permutation
mcp = None
c_mcp = 0
l = [i for i in range(1, len(candidates) + 1)]
for perm in perms:
    temp = {}
    i = 1
    for candidate in perm:
        temp[candidate] = i
        i += 1
    c = 0
    for b in ballots:
        b_num = []
        for candidate in b:
            b_num.append(temp[candidate])
        if (is_consistant(b_num, l)):
            c += ballots[b]
    if mcp is None or c > c_mcp:
        c_mcp = c
        mcp = perm
    print(perm, " :  ", c/sum(ballots.values()))
print("gamma: ", c_mcp/sum(ballots.values()))
print("mcp: ", mcp)
positions = {}
temp = {}
i = 1
for candidate in mcp:
    temp[candidate] = i
    i += 1
c = 0
ballot_index = {}
index = 1
for b in ballots:
    ballot = []
    for candidate in b:
        ballot.append(temp[candidate])
    if len(ballot) == 0 or is_consistant(ballot, mcp) is False:
        continue

    if b not in ballot_index:
        ballot_index[b] = index 
        index += 1

    x = 0 
    v = 0.25
    i = 1
    while (i < len(ballot)):
        if ballot[i] < ballot[i - 1]:
            x -= (v * min(abs(ballot[i] - ballot[0]), abs(ballot[i] - ballot[i - 1])))
        else:
            x += (v * min(abs(ballot[i] - ballot[0]), abs(ballot[i] - ballot[i - 1])))
        v *= 0.5
        i += 1
    x += ballot[0]
    if x not in positions:
        positions[x] = 0
    print(b, " : ", x)
    positions[x] += ballots[b]
p_df = pd.DataFrame(columns=["x", "y"])
for p in positions:
    p_df.loc[len(p_df)] = [p, positions[p]]

sns.lineplot(data=p_df, x="x", y="y")


In [ ]:
plt.figure(figsize=(10, 6))

for x, y in positions.items():
    plt.scatter(x, y)

In [ ]:
import seaborn as sns

filename = "dataverse_files/Maine_11082022_CongressionalDistrict2.csv"
ballots, candidates = parser.parser(filename)

perms = list(itertools.permutations(candidates))
#most consistant permutation
mcp = None
c_mcp = 0
l = [i for i in range(1, len(candidates) + 1)]
for perm in perms:
    temp = {}
    i = 1
    for candidate in perm:
        temp[candidate] = i
        i += 1
    c = 0
    for b in ballots:
        b_num = []
        for candidate in b:
            b_num.append(temp[candidate])
        if (is_consistant(b_num, l)):
            c += ballots[b]
    if mcp is None or c > c_mcp:
        c_mcp = c
        mcp = perm
    print(perm, " :  ", c/sum(ballots.values()))
print("gamma: ", c_mcp/sum(ballots.values()))
print("mcp: ", mcp)
positions = {}
temp = {}
i = 1
for candidate in mcp:
    temp[candidate] = i
    i += 1
c = 0
ballot_index = {}
index = 1
for b in ballots:
    ballot = []
    for candidate in b:
        ballot.append(temp[candidate])
    if len(ballot) == 0 or is_consistant(ballot, mcp) is False:
        continue

    if b not in ballot_index:
        ballot_index[b] = index 
        index += 1

    x = 0 
    v = 0.25
    i = 1
    while (i < len(ballot)):
        if ballot[i] < ballot[i - 1]:
            x -= (v * min(abs(ballot[i] - ballot[0]), abs(ballot[i] - ballot[i - 1])))
        else:
            x += (v * min(abs(ballot[i] - ballot[0]), abs(ballot[i] - ballot[i - 1])))
        v *= 0.5
        i += 1
    x += ballot[0]
    if x not in positions:
        positions[x] = 0
    print(b, " : ", x)
    positions[x] += ballots[b]
p_df = pd.DataFrame(columns=["x", "y"])
for p in positions:
    p_df.loc[len(p_df)] = [p, positions[p]]

sns.lineplot(data=p_df, x="x", y="y")
